In [1]:
# Core
import numpy as np
import pandas as pd

# DB
import sqlalchemy
from sqlalchemy import create_engine

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from ztf_data_load import db_engine, load_ztf_classes, load_ztf_objects, load_ztf_detections
from db_utils import sp2df

In [3]:
df_cls = load_ztf_classes()

In [4]:
df_cls

,ObjectClassID,ObjectClassName
0,0,Other
1,1,Ceph
2,2,DSCT
3,3,EB
4,4,LPV
5,5,RRL
6,6,SNe
7,7,AGN-I
8,8,Blazar
9,9,CV/Nova


In [5]:
df_obj = load_ztf_objects(0, 1000, 'ZTF17')


In [6]:
df_obj

,ObjectCD,ObservationCount,mjd0,mjd1,MeanRA,MeanDEC,MeanMag_g,MeanMag_r,MeanMagPSF_g,MeanMagPSF_r,ObjectClassID,ClassificationProb
0,ZTF17aaaaaal,108,58367.465868,59294.196181,59.899024,50.278084,16.453223,15.666678,16.452571,15.676420,20.0,0.991538
1,ZTF17aaaaaam,71,58455.358530,59296.163125,60.275189,49.811787,18.715262,16.545359,18.763388,16.566218,20.0,0.937527
2,ZTF17aaaaaan,224,58343.446620,59296.196586,64.334171,45.594256,16.543602,16.052844,16.546283,16.052882,20.0,0.869584
3,ZTF17aaaaaao,139,58340.493391,59294.196181,63.989763,45.430490,17.023956,16.093108,17.035832,16.085951,20.0,0.799161
4,ZTF17aaaaaas,200,58348.490567,59292.158542,68.578095,49.087084,17.309472,16.562732,17.320121,16.566059,18.0,0.562905
...,...,...,...,...,...,...,...,...,...,...,...,...
995,ZTF17aaaafxh,501,58344.475255,59296.222755,82.629664,32.175141,17.291135,16.237717,17.304867,16.235135,20.0,0.919372
996,ZTF17aaaafxk,175,58347.448368,59292.206030,82.492257,32.131850,17.856744,16.927300,17.866957,16.936085,20.0,0.865683
997,ZTF17aaaafxt,2,58660.448403,58829.312546,33.720315,30.026622,16.310497,14.205728,16.478888,14.231841,20.0,0.575160
998,ZTF17aaaafxv,3,58721.456852,59127.405174,27.846646,30.382350,15.761253,14.454487,15.703148,14.513006,20.0,0.955649


In [7]:
df_obj.iloc[105:110]

,ObjectCD,ObservationCount,mjd0,mjd1,MeanRA,MeanDEC,MeanMag_g,MeanMag_r,MeanMagPSF_g,MeanMagPSF_r,ObjectClassID,ClassificationProb
105,ZTF17aaaaalg,14,58315.404676,59210.110914,322.216570,45.588921,16.580629,15.774332,16.604777,15.784315,20.0,0.978871
106,ZTF17aaaaalh,135,58288.455069,59219.115637,322.684860,45.301219,17.010044,16.349877,17.037022,16.358285,18.0,0.771089
107,ZTF17aaaaali,28,58293.476667,59202.160023,322.781628,45.454687,16.840763,15.912118,16.846028,15.919175,20.0,0.612402
108,ZTF17aaaaall,92,58290.424907,59223.139549,320.596098,48.335998,16.874866,15.621473,16.873303,15.648763,20.0,0.930158
109,ZTF17aaaaalm,17,58372.310706,59028.468403,320.470442,48.492629,-1.000000,16.133761,-1.000000,16.144079,20.0,0.965826


In [8]:
df_obj.iloc[105:110].fillna(-1.0)

,ObjectCD,ObservationCount,mjd0,mjd1,MeanRA,MeanDEC,MeanMag_g,MeanMag_r,MeanMagPSF_g,MeanMagPSF_r,ObjectClassID,ClassificationProb
105,ZTF17aaaaalg,14,58315.404676,59210.110914,322.216570,45.588921,16.580629,15.774332,16.604777,15.784315,20.0,0.978871
106,ZTF17aaaaalh,135,58288.455069,59219.115637,322.684860,45.301219,17.010044,16.349877,17.037022,16.358285,18.0,0.771089
107,ZTF17aaaaali,28,58293.476667,59202.160023,322.781628,45.454687,16.840763,15.912118,16.846028,15.919175,20.0,0.612402
108,ZTF17aaaaall,92,58290.424907,59223.139549,320.596098,48.335998,16.874866,15.621473,16.873303,15.648763,20.0,0.930158
109,ZTF17aaaaalm,17,58372.310706,59028.468403,320.470442,48.492629,-1.000000,16.133761,-1.000000,16.144079,20.0,0.965826


In [9]:
np.sum(np.isnan(df_obj.MeanMag_g))

0

In [10]:
df_det = load_ztf_detections(0, 10, 0)

In [11]:
df_det

,DetectionID,ObjectCD,mjd,RA,DEC,MagPSF,MagApp,MagNR,Sigma_RA,Sigma_DEC
0,516167660015015006,ZTF17aacnkaw,58270.167662,197.497479,9.661561,17.014763,17.028400,15.795000,0.09,0.09
1,516167660015015009,ZTF18aaxlvxz,58270.167662,197.528147,9.462841,19.939310,20.005800,23.004999,0.09,0.09
2,516167660015015010,ZTF18aaxlvxv,58270.167662,197.125117,9.456218,19.402718,19.185200,22.546000,0.09,0.09
3,516167660015015014,ZTF18aaxlvxx,58270.167662,196.874981,9.368209,19.708012,19.778099,17.354000,0.09,0.09
4,516167660015015017,ZTF18aaxlvxy,58270.167662,196.790613,9.262449,19.971024,19.809299,22.490999,0.09,0.09
5,516167660015015018,ZTF18aavirxa,58270.167662,197.334139,9.278298,19.650688,19.304800,20.801001,0.09,0.09
6,516167660015015020,ZTF18aaxlvxu,58270.167662,197.480780,9.145926,18.448275,18.432600,22.834000,0.09,0.09
7,516167660015015021,ZTF18aakcrxf,58270.167662,196.768319,9.167813,19.312817,19.077600,18.063000,0.09,0.09
8,516167660015015022,ZTF18aaxlvxw,58270.167662,197.446344,9.069189,19.678944,19.318001,22.094999,0.09,0.09
9,516167660115010000,ZTF18aaxlwbm,58270.167662,196.384799,9.343527,20.169724,19.919201,18.766001,0.09,0.09


In [12]:
df_det.dtypes

DetectionID      int64
ObjectCD        object
mjd            float64
RA             float64
DEC            float64
MagPSF         float64
MagApp         float64
MagNR          float64
Sigma_RA       float64
Sigma_DEC      float64
dtype: object